In [6]:
import pandas as pd
# import geopandas as gpd

In [7]:
resta_inspect_df = pd.read_csv(
    filepath_or_buffer= "data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv.zip",
    compression="zip",
    usecols=["CAMIS", "STREET", "ZIPCODE"],
    skip_blank_lines=True,
    on_bad_lines="warn",
)
resta_inspect_df.head()

,CAMIS,STREET,ZIPCODE
0,40511702,SECOND AVENUE,10016.0
1,40511702,SECOND AVENUE,10016.0
2,50046354,BROADWAY,11106.0
3,50061389,HOLDEN BLVD,10314.0
4,41516263,5 AVENUE,11209.0
